# Submission .csv maker

In [2]:
# External Libraries
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch.optim as optim
import torchvision.transforms as transforms
from resnet import ResNet
from utils import load_checkpoint
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Initializing Our Model (implementation [here](resnet.py))**

In [3]:
num_blocks = [2,2,2,2]
model = ResNet(num_blocks=num_blocks).to(DEVICE)


**Number of Parameters in our Model**

In [4]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 269722


In [6]:
def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

cifar10_batch = load_cifar_batch('cifar_test_nolabels.pkl')
images = cifar10_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) 

In [7]:
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
starting_epoch = load_checkpoint(model, optimizer, "eighty_seven.tar") 

In [140]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
labels_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
guess = []
for i in range(10000):
    image = images[i]
    model.eval()
    m = nn.Softmax(dim=1)
    with torch.no_grad():
        x = test_transform(image)
        x = torch.unsqueeze(x,0)
        y = model(x)
        output = m(y)
        predicted_class = output.argmax(dim=1)
        guess.append(predicted_class.item())
        # print(f"Predicted class: {labels_names[predicted_class.item()]}")


In [128]:
import csv
with open("test2", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["ID","Labels"])
    for i, num in enumerate(guess):
        writer.writerow([i,num])